In [37]:
import pandas as pd
import numpy as np
import re
import time
import psutil
import os
from transformers import pipeline
import json
import gzip

In [39]:
df = pd.read_csv("./result_df_before_summary.csv")
df['date'] = pd.to_datetime(df['unix_timestamp'], unit='s').dt.date
dff = df[["request_id","query","question","date","texts"]]
df1 = dff[dff['request_id'].str.startswith("CrisisFACTS-001-")]

bart_summarizer = pipeline("summarization", model="facebook/bart-large-xsum")
pega_summarizer = pipeline("summarization", model="google/pegasus-xsum")

FileNotFoundError: [Errno 2] No such file or directory: './result_df_before_summary.csv'

In [15]:
from joblib import Parallel, delayed
import multiprocessing

num_cores = multiprocessing.cpu_count()
num_batches = num_cores * 2

# Split the dataset into batches
batches = np.array_split(df1['texts'], num_batches)

def clean_and_summarize(text, summarizer):
    cleaned_txt = re.sub(r'#\S+|https?://\S+|@\S+', '', text)
    summary = summarizer(cleaned_txt, max_length=52, min_length=30)
    return summary[0]['summary_text']
    
def clean_and_summarize_parallel(texts):
    return [clean_and_summarize(text, bart_summarizer) for text in texts]
    
# Apply parallel processing
results = Parallel(n_jobs=num_cores)(
    delayed(clean_and_summarize_parallel)(batch.tolist()) for batch in batches
)

# Combine results back into the DataFrame
df1['bart_summary'] = [summary for batch_result in results for summary in batch_result]

Your max_length is set to 52, but your input_length is only 47. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=23)
Your max_length is set to 52, but your input_length is only 20. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=10)
Your max_length is set to 52, but your input_length is only 23. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=11)
Your max_length is set to 52, but your input_length is only 23. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=11)
Your max

In [21]:
df["summary"] = df1["bart_summary"]

In [34]:
with open("my_submission_bart_detail.json", 'w') as f:
    for index, row in df.iterrows():
        fact_ = {
            "requestID": row['request_id'],
            "factText": row['summary'],
            "unixTimestamp": int(row['unix_timestamp']),
            "importance": float(row['avg_importance']),
            "sources": row['docno_list'],
            "streamID": None,
            "informationNeeds": row['q_id']
        }
        
        # Write each dictionary as a separate JSON object on a new line
        json.dump(fact_, f)
        f.write("\n")  # Add a newline after each JSON object

In [35]:
# df.to_csv('bart.csv', index=False)

In [36]:
# Define the input and output folder paths
input_folder = "./submission_json"
output_folder = "./submissions"

# Ensure the output folder exists
os.makedirs(output_folder, exist_ok=True)

# Loop through all files in the input folder
for file_name in os.listdir(input_folder):
    input_file_path = os.path.join(input_folder, file_name)
    output_file_path = os.path.join(output_folder, f"{file_name}.gz")

    # Check if the current item is a file (not a folder)
    if os.path.isfile(input_file_path):
        # Open the input file and compress it into the output folder
        with open(input_file_path, "rb") as f_in:
            with gzip.open(output_file_path, "wb") as f_out:
                shutil.copyfileobj(f_in, f_out)
        print(f"Compressed: {input_file_path} -> {output_file_path}")

FileNotFoundError: [Errno 2] No such file or directory: './submission_json'